In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
import os
import sys

drive.mount('/content/drive', force_remount=True)

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "cs231n/project/"
assert FOLDERNAME is not None, "[!] Enter the foldername."
PROJECT_PATH = f"/content/drive/My Drive/{FOLDERNAME}"
sys.path.append(PROJECT_PATH)

# Change working directory
os.chdir(PROJECT_PATH)

# Confirm
print("✅ Current working directory:", os.getcwd())
print("📁 Contents:", os.listdir('.'))

# 💡 LoRA Training on Stable Diffusion
This notebook trains custom LoRA adapters on Stable Diffusion using your own image-caption pairs.


In [ ]:
!pip install -q diffusers transformers accelerate torchvision safetensors kornia wandb

## Inference Stable diffusion model to sample images with the same prompts as base lines.

In [ ]:
import wandb

import os
os.environ["WANDB_API_KEY"] = "d949a6ce67b5e1b2d44dc67ccbdbc699c95b0fcb"
wandb.login()

wandb.init(
    project="stable-diffusion-calligraphy",
    name="base line generated files",  # give each experiment a unique name
)

In [ ]:
# Loads the CLIP model and processor
from clip import calculate_clip_score
from transformers import CLIPProcessor, CLIPModel

device = "cuda"
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name).to(device)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

In [ ]:
#this block attempts to generate images without fine tuning.

import os
import torch
import wandb
from diffusers import StableDiffusionPipeline

device = "cuda"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float32).to(device)
pipe.unet.eval()

output_dir = "base_line_imgs"
os.makedirs(output_dir, exist_ok=True)

wandb.init(project="calligraphy-generation", name="inference-outputs")

# Define prompts
prompts = [
    "mifu chinese caligraphy xingshu",
    "chu suilong chinese caligraphy kaishu",
    "chu suiliang caligraphy chinese xinshu",
    "deng shiru calligraphy chinese lishu",
    "chinese calligraphy",
    "米芾 书法",
    "褚遂良 书法",
    "书法",
    "text",    # irrelevant prompt
    "Sunset"   # irrelevant prompt
]

sampled_images = []
for prompt in prompts:
    for i in range(3):
        image = pipe(prompt, num_inference_steps=30).images[0]
        wandb.log({"output image": wandb.Image(image, caption=prompt)})
        image_path = f"{output_dir}/{prompt.replace(' ', '_')}_generated_{i}.png"
        image.save(image_path)
        image.show()
        display(image)

# Log all sampled images at once
wandb.log({"samples": sampled_images})

In [ ]:
wandb.finish()